# Eat Safe, Love

## Notebook Set Up

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
collections_list = db.list_collection_names()

print("Collections in the 'uk_food' database:")
pprint(collections_list)

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20

query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
count_result = db.establishments.count_documents(query)
print(f"Number of establishments with a hygiene score of 20: {count_result}")

# Display the first document in the results using pprint
result_document = db.establishments.find_one(query)
print("First document with a hygiene score of 20:")
pprint(result_document)

In [ ]:
# Convert the result to a Pandas DataFrame

df_result = pd.DataFrame(list(db.establishments.find(query)))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df_result)}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df_result.head(10))

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.

query = {"LocalAuthorityName": "London", "RatingValue": {"$gte": 4}}

# Use count_documents to display the number of documents in the result
count_result = db.establishments.count_documents(query)
print(f"Number of establishments in London with RatingValue greater than or equal to 4: {count_result}")

# Display the first document in the results using pprint
result_document = db.establishments.find_one(query)
print("First document in London with RatingValue greater than or equal to 4:")
pprint(result_document)

In [ ]:
# Convert the result to a Pandas DataFrame

df_result_london = pd.DataFrame(list(db.establishments.find(query)))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame for establishments in London with RatingValue >= 4: {len(df_result_london)}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame for establishments in London with RatingValue >= 4:")
print(df_result_london.head(10))

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.

degree_search = 0.01
latitude = 40.7128 
longitude = -74.0060 

# Rating value must equal 5
query = {
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
    "RatingValue": 5
}

# Sort by hygiene score
sort = [("scores.Hygiene", 1)]  
# Find and sort the establishments
result = db.establishments.find(query).sort(sort)

# Print the results
print("Establishments within 0.01 degree of the specified latitude and longitude, with RatingValue equal to 5, sorted by hygiene score:")
for doc in result:
    pprint(doc)

In [ ]:
# Convert result to Pandas DataFrame
df_result_query = pd.DataFrame(list(result))


print(f"Number of rows in the DataFrame for the query result: {len(df_result_query)}")


print("First 10 rows of the DataFrame for the query result:")
print(df_result_query.head(10))

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results
pipeline = [
    {"$match": {"scores.Hygiene": 0}},  # Match establishments with a hygiene score of 0
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},  # Group by Local Authority and count the documents
    {"$sort": SON([("count", -1)])}  # Sort the matches from highest to lowest
]

# Execute the pipeline
result_aggregation = list(db.establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f"Number of documents in the result of the aggregation pipeline: {len(result_aggregation)}")

# Print the first 10 results
print("First 10 results of the aggregation pipeline:")
pprint(result_aggregation[:10])

In [ ]:
# Convert the result to a Pandas DataFrame

df_result_aggregation = pd.DataFrame(result_aggregation)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame for the aggregation pipeline result: {len(df_result_aggregation)}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame for the aggregation pipeline result:")
print(df_result_aggregation.head(10))